STEP-ONE: IMPORT LIBRARIES

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import itertools

STEP-TWO: LOAD DATASET

In [4]:
#load dataset
data = np.load('../ORL_faces.npz') 

# load the "Train Images"
x_train = data['trainX']
#normalize every image
x_train = np.array(x_train,dtype='float32')/255

x_test = data['testX']
x_test = np.array(x_test,dtype='float32')/255

# load the Label of Images
y_train= data['trainY']
y_test= data['testY']

# show the train and test Data format
print('x_train : {}'.format(x_train[:]))
print('Y-train shape: {}'.format(y_train))

x_train : [[0.1882353  0.19215687 0.1764706  ... 0.18431373 0.18039216 0.18039216]
 [0.23529412 0.23529412 0.24313726 ... 0.1254902  0.13333334 0.13333334]
 [0.15294118 0.17254902 0.20784314 ... 0.11372549 0.10196079 0.11372549]
 ...
 [0.44705883 0.45882353 0.44705883 ... 0.38431373 0.3764706  0.38431373]
 [0.4117647  0.4117647  0.41960785 ... 0.21176471 0.18431373 0.16078432]
 [0.45490196 0.44705883 0.45882353 ... 0.37254903 0.39215687 0.39607844]]
Y-train shape: [ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1
  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7
  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13
 14 14 14 14 14 14 1

STEP-THREE: DATA VALIDATION AND TRAINING

In [5]:
x_train, x_valid, y_train, y_valid= train_test_split(
    x_train, y_train, test_size=.05, random_state=1234,)

STEP-FOUR: CHANGE IMAGE SIZE FOR CNN

In [6]:
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)

#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)

print('x_train shape: {}'.format(y_train.shape[0]))
print('x_test shape: {}'.format(y_test.shape))

x_train shape: 228
x_test shape: (160,)


STEP-FIVE: BUILD CNN

In [7]:

#filters= the depth of output image or kernels

cnn_model= Sequential([
    Conv2D(filters=64, kernel_size=(7, 7), activation='relu', padding='same', input_shape= im_shape),
    Conv2D(filters=64, kernel_size=(7, 7), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Conv2D(filters=128, kernel_size=(7, 7), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(7, 7), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Conv2D(filters=256, kernel_size=(7, 7), activation='relu', padding='same'),
    Conv2D(filters=256, kernel_size=(7, 7), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
#     Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),
#     Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Flatten(),
    Dense(2024, activation='relu'),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(20, activation='softmax')  
])

# cnn_model= Sequential([
#     Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
#     MaxPooling2D(pool_size=2),
#     Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
#     MaxPooling2D(pool_size=2),
#     Flatten(),
#     Dense(2024, activation='relu'),
#      Dropout(0.5),
#     Dense(1024, activation='relu'),
#     Dropout(0.5),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     #20 is the number of outputs
#     Dense(20, activation='softmax')  
# ])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

In [8]:
# show the CNN model
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 92, 64)       3200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 92, 64)       200768    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 46, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 46, 128)       401536    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 46, 128)       802944    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 23, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 23, 256)       1

STEP-SIX: TRAIN THE MODEL

In [ ]:
history=cnn_model.fit(
    np.array(x_train), np.array(y_train), batch_size=512,
    epochs=310, verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)),
)

Epoch 1/310
1/1 - 1s - loss: 2.9950 - accuracy: 0.0307 - val_loss: 2.9928 - val_accuracy: 0.1667
Epoch 2/310
1/1 - 1s - loss: 2.9941 - accuracy: 0.0439 - val_loss: 3.0042 - val_accuracy: 0.0000e+00
Epoch 3/310
1/1 - 1s - loss: 2.9940 - accuracy: 0.0570 - val_loss: 3.0084 - val_accuracy: 0.0000e+00
Epoch 4/310
1/1 - 1s - loss: 2.9968 - accuracy: 0.0307 - val_loss: 3.0119 - val_accuracy: 0.0000e+00
Epoch 5/310
1/1 - 1s - loss: 3.0005 - accuracy: 0.0702 - val_loss: 3.0093 - val_accuracy: 0.0000e+00
Epoch 6/310
1/1 - 1s - loss: 2.9925 - accuracy: 0.0614 - val_loss: 3.0082 - val_accuracy: 0.0000e+00
Epoch 7/310
1/1 - 1s - loss: 2.9907 - accuracy: 0.0439 - val_loss: 3.0089 - val_accuracy: 0.0000e+00
Epoch 8/310
1/1 - 1s - loss: 2.9882 - accuracy: 0.0658 - val_loss: 3.0123 - val_accuracy: 0.0000e+00
Epoch 9/310
1/1 - 1s - loss: 2.9934 - accuracy: 0.0614 - val_loss: 3.0154 - val_accuracy: 0.0000e+00
Epoch 10/310
1/1 - 1s - loss: 2.9955 - accuracy: 0.0395 - val_loss: 3.0152 - val_accuracy: 0.00

Epoch 84/310
1/1 - 1s - loss: 0.1457 - accuracy: 0.9605 - val_loss: 0.1112 - val_accuracy: 1.0000
Epoch 85/310
1/1 - 1s - loss: 0.1434 - accuracy: 0.9474 - val_loss: 0.1014 - val_accuracy: 1.0000
Epoch 86/310
1/1 - 1s - loss: 0.1865 - accuracy: 0.9430 - val_loss: 0.0954 - val_accuracy: 1.0000
Epoch 87/310
1/1 - 1s - loss: 0.1542 - accuracy: 0.9386 - val_loss: 0.0865 - val_accuracy: 1.0000
Epoch 88/310
1/1 - 1s - loss: 0.1604 - accuracy: 0.9430 - val_loss: 0.0736 - val_accuracy: 1.0000
Epoch 89/310
1/1 - 1s - loss: 0.1363 - accuracy: 0.9474 - val_loss: 0.0796 - val_accuracy: 1.0000
Epoch 90/310


In [ ]:
# Evaluating test data
scor = cnn_model.evaluate( np.array(x_test),  np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

STEP-SEVEN: PLOT RESULTS

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

STEP-EIGHT: PLOT CONFUSION MATRIX

In [ ]:
predicted =np.array( cnn_model.predict(x_test))
#print(predicted)
#print(y_test)
ynew = cnn_model.predict_classes(x_test)


Acc=accuracy_score(y_test, ynew)
print("accuracy : ")
print(Acc)
#/tn, fp, fn, tp = confusion_matrix(np.array(y_test), ynew).ravel()
cnf_matrix=confusion_matrix(np.array(y_test), ynew)

y_test1 = np_utils.to_categorical(y_test, 20)



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


print('Confusion matrix, without normalization')
print(cnf_matrix)

plt.figure()
plot_confusion_matrix(cnf_matrix[1:10,1:10], classes=[0,1,2,3,4,5,6,7,8,9],
                      title='Confusion matrix, without normalization')

plt.figure()
plot_confusion_matrix(cnf_matrix[11:20,11:20], classes=[10,11,12,13,14,15,16,17,18,19],
                      title='Confusion matrix, without normalization')

print("Confusion matrix:\n%s" % confusion_matrix(np.array(y_test), ynew))
print(classification_report(np.array(y_test), ynew))

In [ ]:
# CPU/GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())